In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Read and merge Census

In [5]:
data_path = "../data_subsidy/"
acs1 = pd.read_csv(data_path + 'ACSDP5Y2016.DP03_data_with_overlays_2022-03-17T201553.csv', na_values=['(X)']) #, encoding='cp1252'
acs2 = pd.read_csv(data_path + 'ACSDP5Y2016.DP05_data_with_overlays_2021-12-09T232536.csv', na_values=['(X)']) 
acs3 = pd.read_csv(data_path + 'ACSST5Y2016.S1901_data_with_overlays_2022-03-17T201851.csv', na_values=['(X)']) 
acs4 = pd.read_csv(data_path + 'ACSST5Y2016.S1701_data_with_overlays_2021-12-12T065723.csv', na_values=['(X)'])


def remove_cols(df):
    df.iloc[0]['GEO_ID'] = 'county_name'
    keywords =["Estimate", "county_name"]
    
    #drop more columns
    clean_df1 = pd.DataFrame(df.iloc[0].str.contains("Estimate|county_name"))
    clean_df2 = pd.DataFrame(df.iloc[0].str.contains("Total|county_name"))
    clean_df = clean_df1 & clean_df2

    clean_df = clean_df.loc[clean_df[0]==True]
    df.drop(df.columns.difference(clean_df.index), 1, inplace=True)
    df['County'] = df['GEO_ID'].str[-5:]
    df['County'] = df['County'].astype(str)  
    df['County'] = df['County']
    df.drop(columns=['GEO_ID'], inplace=True)
    df.fillna(0, inplace = True)
    return df
dfs = [acs1, acs2, acs3, acs4]
for i in dfs:
    remove_cols(i)
    
for i in dfs:
    print(i.shape)

(3221, 14)
(3221, 31)
(3221, 5)
(3221, 62)


In [6]:
acs1 = acs1.loc[1:]
acs2 = acs2.loc[1:]
acs3 = acs3.loc[1:]
acs4 = acs4.loc[1:]

In [7]:
from functools import partial, reduce
dfs = [acs1, acs2, acs3, acs4]
merge = partial(pd.merge, on=['County'], how='outer')
all_acs = reduce(merge, dfs)
all_acs

,DP03_0051E,DP03_0052E,DP03_0053E,DP03_0054E,DP03_0055E,DP03_0056E,DP03_0057E,DP03_0058E,DP03_0059E,DP03_0060E,...,S1701_C01_052E,S1701_C01_053E,S1701_C01_054E,S1701_C01_055E,S1701_C01_056E,S1701_C01_057E,S1701_C01_058E,S1701_C01_059E,S1701_C01_060E,S1701_C01_061E
0,20800,1254,1337,1928,2022,3118,4064,2751,3048,794,...,952,860,1467,1336,1152,1091,5417,3374,1132,3316
1,75149,4530,4226,8765,8019,10905,13942,9008,9532,2957,...,4493,3936,5446,6397,5225,5209,6288,12802,6793,14202
2,9122,1326,896,1409,1021,1365,1335,859,724,140,...,651,364,606,655,733,674,5349,1274,621,1993
3,7048,601,382,1183,1012,1069,1249,806,534,157,...,294,308,561,323,484,380,7539,742,317,1504
4,20619,1662,1262,2223,2756,3190,4501,2275,1966,537,...,894,704,1072,1267,1257,1215,6610,2121,1007,3631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,17356,5750,2027,2962,1874,1748,2062,544,273,63,...,639,757,1138,1288,1379,1140,7833,1374,483,4669
3216,2673,618,397,617,445,420,132,44,0,0,...,41,230,273,319,374,232,6304,495,251,751
3217,7870,2192,921,1597,1267,992,588,218,84,11,...,430,303,470,496,547,319,7023,489,531,1656
3218,12011,3849,1817,2202,1677,1329,827,170,105,26,...,513,857,961,1249,1165,661,7824,937,418,4391


# Preprocess CSR

In [8]:
csr = pd.read_csv(data_path + 'csrzipcounty2016.csv', na_values=['.'])
csr.columns = csr.iloc[2]
csr = csr.iloc[4:-1]
csr.reset_index(drop=True)
csr

2,State,FIPS County Code,County Name,Total Number of Consumers,Number of Consumers with CSR (AV of 73%/87%/94%),Number of Consumers with CSR AV of 73%,Number of Consumers with CSR AV of 87%,Number of Consumers with CSR AV of 94%,Average Monthly Advanced CSR Payment for Consumers with 73%,Average Monthly Advanced CSR Payment for Consumers with 87%,Average Monthly Advanced CSR Payment for Consumers with 94%
4,AK,02013,ALEUTIANS EAST,38,16,NaN,NaN,NaN,NaN,NaN,NaN
5,AK,02016,ALEUTIANS WEST,38,16,NaN,NaN,NaN,NaN,NaN,NaN
6,AK,02020,ANCHORAGE,"9,484","3,917",787,"1,640","1,490",$26,$204,$290
7,AK,02050,BETHEL,73,18,NaN,11,NaN,NaN,$234,NaN
8,AK,02060,BRISTOL BAY BOROUGH,34,12,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2600,WY,56037,SWEETWATER,"1,168",665,109,222,334,$16,$119,$173
2601,WY,56039,TETON,"2,812","1,411",428,484,499,$15,$115,$165
2602,WY,56041,UINTA,663,356,79,116,161,$16,$133,$181
2603,WY,56043,WASHAKIE,341,173,33,54,86,$15,$138,$192


In [9]:
csr_clean = csr.copy()
csr_clean = csr_clean
for column in list(csr.columns)[3:]:
    csr_clean[column] = csr_clean[column].fillna('0')
    csr_clean[column] = csr_clean[column].apply(lambda x: float((str(x)).replace('$','').replace(',','')) )

In [10]:
merged_acs = all_acs.merge(csr_clean, how='left', left_on='County', right_on='FIPS County Code')
merged_acs

,DP03_0051E,DP03_0052E,DP03_0053E,DP03_0054E,DP03_0055E,DP03_0056E,DP03_0057E,DP03_0058E,DP03_0059E,DP03_0060E,...,FIPS County Code,County Name,Total Number of Consumers,Number of Consumers with CSR (AV of 73%/87%/94%),Number of Consumers with CSR AV of 73%,Number of Consumers with CSR AV of 87%,Number of Consumers with CSR AV of 94%,Average Monthly Advanced CSR Payment for Consumers with 73%,Average Monthly Advanced CSR Payment for Consumers with 87%,Average Monthly Advanced CSR Payment for Consumers with 94%
0,20800,1254,1337,1928,2022,3118,4064,2751,3048,794,...,01001,AUTAUGA,1896.0,1341.0,142.0,399.0,800.0,13.0,87.0,117.0
1,75149,4530,4226,8765,8019,10905,13942,9008,9532,2957,...,01003,BALDWIN,11778.0,8342.0,1246.0,2426.0,4670.0,13.0,91.0,121.0
2,9122,1326,896,1409,1021,1365,1335,859,724,140,...,01005,BARBOUR,801.0,584.0,62.0,155.0,367.0,13.0,98.0,132.0
3,7048,601,382,1183,1012,1069,1249,806,534,157,...,01007,BIBB,629.0,477.0,58.0,121.0,298.0,15.0,101.0,123.0
4,20619,1662,1262,2223,2756,3190,4501,2275,1966,537,...,01009,BLOUNT,2264.0,1663.0,197.0,437.0,1029.0,14.0,96.0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,17356,5750,2027,2962,1874,1748,2062,544,273,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3216,2673,618,397,617,445,420,132,44,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,7870,2192,921,1597,1267,992,588,218,84,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,12011,3849,1817,2202,1677,1329,827,170,105,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
missing_value_df = pd.DataFrame(merged_acs.isnull().sum())
missing_value_df.T #= missing_value_df.T
#all_merged = pd.concat([merged_acs, missing_value_df])

,DP03_0051E,DP03_0052E,DP03_0053E,DP03_0054E,DP03_0055E,DP03_0056E,DP03_0057E,DP03_0058E,DP03_0059E,DP03_0060E,...,FIPS County Code,County Name,Total Number of Consumers,Number of Consumers with CSR (AV of 73%/87%/94%),Number of Consumers with CSR AV of 73%,Number of Consumers with CSR AV of 87%,Number of Consumers with CSR AV of 94%,Average Monthly Advanced CSR Payment for Consumers with 73%,Average Monthly Advanced CSR Payment for Consumers with 87%,Average Monthly Advanced CSR Payment for Consumers with 94%
0,0,0,0,0,0,0,0,0,0,0,...,622,622,622,622,622,622,622,622,622,622


In [64]:
merged_acs.fillna(0, inplace = True)
merged_acs.to_csv('county_characteristics.csv', index=False)